In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from tqdm.auto import tqdm

# Call sys.path.append only once
# import os, sys; 
# sys.path.append(os.path.dirname(os.path.realpath('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/Neuro-symbolic-AI/column.py')))

# sys.path.remove(os.path.dirname(os.path.realpath('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/Neuro-symbolic-AI/column.py')))
import column


tqdm.pandas()

In [ ]:
print(sys.path)

In [ ]:
# df = pd.read_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_features_2016_2020_v2.parquet.gzip')
df = pd.read_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_features_2016_2020_v3.parquet.gzip')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df=df.rename(columns = {'band_x':'band', 'vg_state_x':'vg_state'})

In [ ]:
# Split the data set and extract time series features
X = df.loc[:,~df.columns.isin(['veranst_segment','vg_inkasso'])]
y = df['vg_inkasso']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [ ]:
display(df.head(5))

In [ ]:
print(len(df.band.unique()))#317011 - unique bands
print(len(df.promoter.unique()))#176456 - unique promoter/organizer
print(len(df.vg_state.unique()))#16 - unique bundesland/states
print(len(df.tarif_bez.unique()))#87 - unique Tarif

In [ ]:
print(len(X_train))

# Extract inkasso distribution with respect to the time of the event

In [ ]:
def extract_features(features, target):
    stat_var = ['band']
    # stat_var = ['vg_state']
    # stat_var = ['promoter']
    # stat_var = ['tarif_bez']
    df_stat = {}

    # 1. Take vg_inkasso value before the current datetime - (split the train and test and calculate the inkasso distribution-after feature calculation)
    # 2. Timestamp as index, and have location/band as columns in df
    # 3. Predefine the single df and iterate over it

    def get_descr_stat(row):
        # row represents each line(row) in the dataframe
        def descr_stat(row, var):
            # Initialize a empty pandas dataframe
            result = pd.Series(dtype='float64')

            if not pd.isnull(row[var]):
                #get vg_inkasso collected for past events(that occured before current date of the row) for each location/state, each band and each promoter
                matching_indexes = features[(features['vg_datum_von']<row['vg_datum_von'])&(features[var] == row[var])].index.values.tolist()

                inkasso =  list(target[target.index.isin(matching_indexes)].values)

                # Returns the mean, count, std, max, min and 0.05, 0.10..0.95 percentiles of the list of vg_inkasso values
                # for a specific attribute like location, band, promotor
                result = pd.Series(inkasso, dtype='float64').describe(percentiles=percentiles)
                result = result.add_prefix('{}_'.format(var))

            result.name = row.name
            return result

        descr_stat_result = pd.Series(dtype='float64')
        for v in stat_var:
            descr_stat_result = pd.concat([descr_stat_result, descr_stat(row, v)])

        return descr_stat_result

 
    
    percentiles = [round(x, 2) for x in np.linspace(0, 1, 21)[1:-1].tolist()]

    # Each row of the dataframe 'df' is passed as row argument for get_descr_stat function
    df_featurize_descr_stat = features.progress_apply(get_descr_stat, axis=1)
    # df_featurize_descr_stat.to_pickle('./data/export_descr_stat_2016_2020.pkl.bz2', protocol=4)

    # display(df_featurize_descr_stat.head(5))
    return df_featurize_descr_stat

# For training data - split the data and extract features
# For testing data - Features can be extracted without splits

# 1393757
# Splitting 1393792 entries to 128 splits, where each split contains 10889 entries
X_train_splits = np.array_split(X_train,200)
y_train_splits = np.array_split(y_train,200)

# X_train_splits = np.array_split(X_test,200)
# y_train_splits = np.array_split(y_test,200)

df_featurize_descr_stat_result = pd.Series(dtype='float64')
for i in range(0,len(X_train_splits)):
    X_train_slice = X_train_splits[i]
    y_train_slice = y_train_splits[i]
    result = extract_features(X_train_slice, y_train_slice)
    df_featurize_descr_stat_result = pd.concat([df_featurize_descr_stat_result, result])

df_featurize_descr_stat_result.index.name = 'ID'
df_featurize_descr_stat_result = df_featurize_descr_stat_result.drop(0, axis=1) # remove 0th index column with NaN values 
display(df_featurize_descr_stat_result.head(5))

# df_featurize_descr_stat = extract_features(X_test, y_test)
# display(df_featurize_descr_stat.head(5))

### Train set

In [ ]:
# Training set - Saving band feature inkasso distribution
# df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [ ]:
# Training set - Saving vg_state feature inkasso distribution
# df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [ ]:
# Training set - Saving promoter feature inkasso distribution
# df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [ ]:
# Training set - Saving Tarif feature inkasso distribution
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_tarif_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

### Test set

In [ ]:
# Testing set - Saving promoter feature inkasso distribution
# df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [ ]:
# Testing set - Saving band feature inkasso distribution
# df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [ ]:
# Testing set - Saving vg_state feature inkasso distribution
# df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [ ]:
# Testing set - Saving Tarif feature inkasso distribution
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_tarif_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

## Merge the extracted features into training set and testing set

In [ ]:
# Merge the time series distribution of vg_inkasso values for training data of band, promoter, vg_state, tarif
# data_path_train_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
# data_path_train_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
# data_path_train_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

data_path_train_vg_state = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_train_band = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_train_promoter = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_train_tarif = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_tarif_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')

if data_path_train_vg_state.exists() and data_path_train_band.exists() and data_path_train_promoter.exists() and data_path_train_tarif.exists():
    df_train_state = pd.read_parquet(data_path_train_vg_state)
    df_train_band = pd.read_parquet(data_path_train_band)
    df_train_promoter = pd.read_parquet(data_path_train_promoter)
    df_train_tarif = pd.read_parquet(data_path_train_tarif)

    df_train_time_series_merged_descr_stat = df_train_state.merge(df_train_band, how='left', on='ID').merge(df_train_promoter, how='left', on='ID').merge(df_train_tarif, 
                                                                                                                                                           how ='left', 
                                                                                                                                                           on='ID')

display(df_train_time_series_merged_descr_stat.head(5))
print(df_train_time_series_merged_descr_stat.shape)

In [ ]:
# Merge the time series distribution of vg_inkasso values for testing data of band, promoter, vg_state, tarif
# data_path_test_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
# data_path_test_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
# data_path_test_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

data_path_test_vg_state = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_test_band = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_test_promoter = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_test_tarif = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_tarif_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')

if data_path_test_vg_state.exists() and data_path_test_band.exists() and data_path_test_promoter.exists() and data_path_test_tarif.exists():
    df_test_state = pd.read_parquet(data_path_test_vg_state)
    df_test_band = pd.read_parquet(data_path_test_band)
    df_test_promoter = pd.read_parquet(data_path_test_promoter)
    df_test_tarif = pd.read_parquet(data_path_test_tarif)

    df_test_time_series_merged_descr_stat = df_test_state.merge(df_test_band, how='left', on='ID').merge(df_test_promoter,
                                                                                                            how='left',
                                                                                                            on='ID').merge(df_test_tarif, how='left', on='ID')

display(df_test_time_series_merged_descr_stat.head(5))
print(df_test_time_series_merged_descr_stat.shape)

In [ ]:
# features = column.features_v3
# v3 features
features = ['place_kirche', 'place_hotel', 'place_cafe', 'place_theater', 'place_club', 'place_halle', 'place_gaststaette', 
'place_festhalle', 'place_kulturzentrum', 'place_festzelt', 'place_schloss', 'place_pub', 'place_stadthalle', 'place_park', 
'place_gasthof', 'place_kabarett', 'place_arena', 'place_schlachthof', 'place_wandelhalle', 'place_turnhalle', 'place_buergerhaus', 'place_museum', 
'place_rathaus', 'place_staatsbad', 'place_zelt', 'place_jazz', 'place_forum', 'place_gymnasium', 'place_schule', 'place_sporthalle', 

'tarif_u-v ii. 1 (+ii 2)', 'tarif_u-k (musiker)', 'tarif_u-k (musiker) mindestverguetung', 'tarif_u-k i (+ ii 2b) - mit sonstigem geldwerten vorteil', 
'tarif_u-st i (musiker) nl', 'tarif_u-v iii. 1', 'tarif_u-v iii. 2', 'tarif_u-k iii. 2d) (musiker) - vor geladenen gaesten', 'tarif_variete i (musiker)', 
'tarif_u-v vi. b', 'tarif_e (musiker)', 'tarif_p-k i. (u-musik)', 'tarif_u-k ii. (musiker) bis 50 min', 'tarif_u-k ii. (musiker) bis 20 min', 'tarif_e-p', 
'tarif_u-k ii. (musiker) bis 15 min', 'tarif_u-k ii. (musiker) bis 25 min', 'tarif_u-k ii. (musiker) bis 30 min', 'tarif_u-st (musiker)', 
'tarif_u-k ii. (musiker) bis 10 min', 'tarif_chorverband konzert u-musik', 'tarif_vk i 3 zirkusunternehmen (musiker)', 'tarif_u-k ii. (musiker) bis 5 min', 
'tarif_u-k ii. (musiker) bis 35 min', 'tarif_u-k ii. (musiker) bis 40 min', 'tarif_p-k i. (e-musik)', 'tarif_u-st i. (musiker)', 'tarif_u-k ii (musiker) mindestverguetung', 
'tarif_u-k ii. (musiker) bis 45 min', 'tarif_u-v vi. b mindestverguetung', 

# 30 bands

'state_bavaria', 'state_rhineland-palatinate', 'state_baden-wuerttemberg', 'state_north rhine-westphalia', 'state_thuringia', 
'state_hesse', 'state_brandenburg', 'state_schleswig-holstein', 'state_berlin', 'state_mecklenburg-western pomerania', 
'state_lower saxony', 'state_hamburg', 'state_saarland', 'state_saxony-anhalt', 'state_saxony', 'state_bremen', 

'vg_datum_year', 'vg_datum_month', 'vg_datum_day_of_week', 'vg_datum_season', 

# 30 promoters

'veranst_segment','vg_inkasso'
]

In [ ]:
df_features = df[features]

In [ ]:
# Train size = 1393757
# Test size = 348440
df_training = df_train_time_series_merged_descr_stat.merge(df_features, how='left', on='ID')
print(df_training.shape)
display(df_training.head(5))

In [ ]:
df_testing = df_test_time_series_merged_descr_stat.merge(df_features, how='left', on='ID')
print(df_testing.shape)
display(df_testing.head(5))

## Allow only maximum of 2 NaN per row

In [ ]:
test_copy = df_testing.copy()

In [ ]:
# Allow only maximum of 2 NaN per row or remove the rows with more than 2 NaN values Eg: band_std = NaN and promoter_std = NaN
test_copy.dropna(axis=0, thresh=152, inplace=True)
display(test_copy.head(5))
#(151287, 154)

In [ ]:
train_copy = df_training.copy()

In [ ]:
# Allow only maximum of 2 NaN per row or remove the rows with more than 2 NaN values Eg: band_std = NaN and promoter_std = NaN
train_copy.dropna(axis=0, thresh=152, inplace=True)
display(train_copy.head(5))
print(train_copy.shape)
#(195254, 154)

## Set NaN values to 0.0 in 'promoter_std', 'band_std', 'vg_state_std' columns

In [ ]:
train_copy['promoter_std'] = np.where((train_copy.promoter_std.isna()),0.0,train_copy.promoter_std)
train_copy['band_std'] = np.where((train_copy.band_std.isna()),0.0,train_copy.band_std)
train_copy['vg_state_std'] = np.where((train_copy.vg_state_std.isna()),0.0,train_copy.vg_state_std)

In [ ]:
# Check is there are any NaN values in any of the columns
train_copy.isna().any()

## Export training and testing dataframe, each containing 238 features

In [ ]:
# df_training.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_training_features_2016_2020.parquet.gzip',compression='gzip')
df_training.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_training_features_2016_2020_v1.parquet.gzip',compression='gzip')

In [ ]:
# df_testing.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_testing_features_2016_2020.parquet.gzip',compression='gzip')
df_testing.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_testing_features_2016_2020_v1.parquet.gzip',compression='gzip')

## Splitting to training and validation sets

In [ ]:
# training, validation = train_test_split(df_training, test_size=0.15,random_state=1)
# print(training)
# print(validation)